### Imports

In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger
from datetime import datetime
import extras.ourUtils
import numpy as np
import Models

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


### Init

In [2]:
batch_size = 40
full_train = True
path = '/home/jaskmo/Documents/programering/02456DomainAdaptation/'
source_data = path + 'taperImages/pysNetData'
target_data = path + 'taperImages/hData'

## Get data as generators

In [3]:
datagen = ImageDataGenerator(rescale=1./255)

# make a joint generator form a Physionet generator and a Hospital generator
def gen_DAnet(source, target, LL=batch_size):
    while True:
        source_data, source_lable = source.next()
        target_data, target_lable = target.next()
        DA_data = np.concatenate((source_data,target_data),axis=0)
        class_lable = np.concatenate((source_lable,target_lable),axis=0)
        
        
        domain_tmp = np.ones(class_lable.shape[0], dtype='int8')
        domain_tmp[int(LL/2):] = domain_tmp[int(LL/2):] * 0
        domain_lable = np.concatenate((domain_tmp.reshape(class_lable.shape[0],1),
                                       np.flip(domain_tmp,0).reshape(class_lable.shape[0],1)),1)

        yield(DA_data, {'lplOut':class_lable,'dplOut':domain_lable})

#### Train data

In [4]:

train_gen_source = datagen.flow_from_directory(source_data + '/train', target_size=(224, 224), 
                                               batch_size=int(np.floor_divide(batch_size,2)), 
                                               class_mode='categorical', shuffle=True)
train_gen_target = datagen.flow_from_directory(target_data + '/train', target_size=(224, 224), 
                                               batch_size=int(np.floor_divide(batch_size,2)), 
                                               class_mode='categorical', shuffle=True)
# init gen_DAnet
train_gen_DAnet = gen_DAnet(source=train_gen_source, target=train_gen_target, LL=batch_size)

train_stepE = np.floor_divide(train_gen_source.n, batch_size)

Found 29772 images belonging to 5 classes.
Found 10629 images belonging to 5 classes.


In [5]:
#data, lab = next(train_gen_DAnet)

In [6]:
#lab.get('lplOut').shape

#### validation data

In [7]:
valid_gen_source = datagen.flow_from_directory(source_data + '/validation', target_size=(224, 224), 
                                               batch_size=int(np.floor_divide(batch_size,2)), 
                                               class_mode='categorical', shuffle=True)
valid_gen_target = datagen.flow_from_directory(target_data + '/validation', target_size=(224, 224), 
                                               batch_size=int(np.floor_divide(batch_size,2)), 
                                               class_mode='categorical', shuffle=True)
# init gen_DAnet
valid_gen_DAnet = gen_DAnet(source=valid_gen_source, target=valid_gen_target, LL=batch_size)

val_stepE = np.floor_divide(valid_gen_source.n, batch_size)

Found 4807 images belonging to 5 classes.
Found 2838 images belonging to 5 classes.


#### test data

In [8]:
test_gen_source = datagen.flow_from_directory(source_data + '/test', target_size=(224, 224), 
                                               batch_size=int(np.floor_divide(batch_size,2)), 
                                               class_mode='categorical', shuffle=True)
test_gen_target = datagen.flow_from_directory(target_data + '/test', target_size=(224, 224), 
                                               batch_size=int(np.floor_divide(batch_size,2)), 
                                               class_mode='categorical', shuffle=True)
# init gen_DAnet
test_gen_DAnet = gen_DAnet(source=test_gen_source, target=test_gen_target, LL=batch_size)

test_stepE = np.floor_divide(test_gen_source.n, batch_size)

Found 3862 images belonging to 5 classes.
Found 2722 images belonging to 5 classes.


### get model

In [9]:
current_model = Models.DA_model(batch_size=batch_size)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image_input (InputLayer)         (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
vgg16 (Model)                    multiple              14714688    image_input[0][0]                
____________________________________________________________________________________________________
preDa (Flatten)                  (None, 25088)         0           vgg16[1][0]                      
____________________________________________________________________________________________________
flipGrad (Lambda)                (None, 25088)         0           preDa[0][0]                      
___________________________________________________________________________________________

### Callbacks

In [10]:
now = datetime.now()
csv_logger = CSVLogger('/media/jaskmo/ELEK/bme/Project02456/trainingLog/DA_Model' + 
                           str(now.day) + '-' + str(now.month) + '-' + str(now.year) + '_' + 
                           str(now.hour) + str(now.minute) + '.log')

### Fit the S!@¤

In [11]:
current_model.fit_generator(test_gen_DAnet, steps_per_epoch=train_stepE, validation_data=valid_gen_DAnet, 
                            validation_steps=val_stepE, epochs=10, verbose=1, callbacks=[csv_logger], 
                            )#class_weight=weights_dic)

Epoch 1/10


InvalidArgumentError: Incompatible shapes: [20] vs. [40]
	 [[Node: loss/lplOut_loss/mul_1 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](loss/lplOut_loss/Mean, _arg_lplOut_sample_weights_0_4/_119)]]

Caused by op 'loss/lplOut_loss/mul_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python3.5/dist-pac